# LinkedIn Post AI Automation

In [ ]:
# Import all packages

! pip install feedparser
! pip install google-generativeai

import feedparser
from datetime import datetime, timedelta
import pandas as pd
import os
import google.generativeai as genai
# from google import genai # Removed this line
import json
import re

## Step 1: Gather Relevant Headlines



#### Publications Sources

##### HR & People-Focused Publications

* **HRM Asia**: Covers HR trends, workplace practices, leadership, and people management across Asia-Pacific.
* **HR Asia**: Focuses on employee engagement, workplace culture, and employer branding across Asian markets.
* **HRD Asia**: Provides HR news, leadership insights, and case studies for HR professionals in Asia.
* **People Matters**: A leading HR and talent management publication in India and Asia, covering people strategy, tech, and leadership.
* **SHRM**: Global HR authority from the U.S., sharing HR policies, compliance, leadership practices, and workplace trends.
* **HRA Magazine (Japan HR Society)**: Japan-focused HR insights, with emphasis on best practices, talent strategy, and HR innovation.
* **The Japan Times – Human Resources**: English-language reporting on HR trends, workplace issues, and employment practices in Japan. **bold text**


##### HR Tech & Business Tech Publications

* **CIO Asia – HR Tech**: Covers the intersection of HR and IT, with insights on HR technology, digital transformation, and workforce analytics.
* **Tech in Asia**: Startup and technology-focused, including coverage on HR tech, talent management, and workplace innovation.
* **e27**: Startup ecosystem and tech innovation in Southeast Asia, often highlighting HR tech solutions and workforce trends.

In [ ]:
# RSS feeds for HR / Talent news
feeds = {
    "HRM Asia": "https://www.hrmasia.com/feed/",
    "HR Asia": "https://hr.asia/feed/",
    "HRD Asia": "https://www.hcamag.com/asia/feed/",
    "People Matters": "https://www.peoplematters.in/feed",
    "SHRM": "https://www.shrm.org/rss/News/rss.xml",
    "HRA Magazine (Japan HR Society)": "https://jhrs.wildapricot.org/hr_agenda/feed",
    "The Japan Times – Human Resources": "https://www.japantimes.co.jp/tag/human-resources/feed/",
    "CIO Asia – HR Tech": "https://www.cio.com/category/hr-it/feed/",
    "Tech in Asia": "https://www.techinasia.com/feed",
    "e27": "https://e27.co/feed/"
}
keywords = ["HR", "Talent", "Recruitment", "Hiring", "Employee", "Workforce", "Human Resources", "AI"]

# Date range: past week
today = datetime.now()
one_week_ago = today - timedelta(days=7)

# Scrape headlines with date, link, and source
scraped_articles_list = []

for source_name, feed_url in feeds.items():
    feed = feedparser.parse(feed_url)
    for entry in feed.entries:
        if hasattr(entry, 'published_parsed'):
            published = datetime(*entry.published_parsed[:6])
            if one_week_ago <= published <= today:
                if any(keyword.lower() in entry.title.lower() for keyword in keywords):
                    scraped_articles_list.append({
                        "Headline": entry.title,
                        "Date": published.strftime("%Y-%m-%d"),
                        "Source": source_name,
                        "Link": entry.link
                    })

# Display in a table
if not scraped_articles_list:
    print("No relevant headlines found this week.")
else:
    scraped_articles_df = pd.DataFrame(scraped_articles_list)
    scraped_articles_df.index += 1  # start numbering at 1
    scraped_articles_df.rename_axis("No.", inplace=True)
    print(f"Found {len(scraped_articles_list)} articles")
    display(scraped_articles_df)

Found 20 articles


,Headline,Date,Source,Link
No.,,,,
1,Beyond disruption: How top CHROs are shaping t...,2025-09-03,HRM Asia,https://hrmasia.com/beyond-disruption-how-top-...
2,Why hiring Gen Z means HR has to rethink the e...,2025-09-01,HRM Asia,https://hrmasia.com/why-hiring-gen-z-means-hr-...
3,Organisations in Singapore brace for slower hi...,2025-08-29,HRM Asia,https://hrmasia.com/organisations-in-singapore...
4,From fixed to flexible: Why Asia’s top talent ...,2025-08-28,HRM Asia,https://hrmasia.com/from-fixed-to-flexible-why...
5,The quiet ambition: How Vietnam is winning AI ...,2025-09-03,e27,https://e27.co/the-quiet-ambition-how-vietnam-...
6,The power of networks: How David Blumberg buil...,2025-09-02,e27,https://e27.co/the-power-of-networks-how-david...
7,Empathy-first algorithms: The marriage of AI a...,2025-09-02,e27,https://e27.co/empathy-first-algorithms-the-ma...
8,Top 5 strategies on how startup founders can d...,2025-09-02,e27,https://e27.co/top-5-strategies-on-how-startup...
9,Beyond the launch: What makes app fail?,2025-09-02,e27,https://e27.co/beyond-the-launch-what-makes-ap...


## Step 2: Use Gemini to choose the Most Engaging Headline





Get API key: https://aistudio.google.com/app/apikey

In [ ]:
# Set your Gemini API key
import google.generativeai as genai
from google.colab import userdata

# Fetch the API key from Colab secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the genai library
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize model (Note: client is not needed for generate_content with the configured genai)
print("Gemini SDK configured successfully!")

# The generate_content function is available directly after configuring genai
# client = genai.Client() # Removed this line as it's not needed for generate_content

SecretNotFoundError: Secret GOOGLE_API_KEY does not exist.

In [ ]:
# Optimized prompt with context and target audience
scoring_headlines_prompt = """Context: You are creating engaging LinkedIn posts for Cygnify, a Talent Acquisition as a Service (TAaaS) company. Cygnify offers unlimited global hires on a fixed monthly subscription, with no placement fees, a plug & play TA team that can onboard in under 5 minutes, and flexible month-to-month plans.

Target audience:
1. Fast-growing startups/scale-ups needing to rapidly build teams globally. They value speed, flexibility, and cost-effective scaling.
2. Mid-to-large enterprises optimizing recruitment costs, seeking predictable expenses and ROI.
3. Companies expanding internationally, needing a partner to navigate global hiring complexities.

Task: Rank the following headlines from most to least engaging for this audience. Assign each headline a numeric score from 0 (not engaging) to 1 (extremely engaging). Return only a JSON object mapping headline to score.

Headlines:
"""

# Append all headlines from the DataFrame
for i, headline in enumerate(scraped_articles_df['Headline'], 1):
    scoring_headlines_prompt += f"{i}. {headline}\n"

# JSON output example to guide the model
scoring_headlines_prompt += "\nJSON output example: {\"Headline text\": 0.75, ...}"

# Call Gemini model
scored_headlines_llm_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=scoring_headlines_prompt
)

# Print LLM output
print(scored_headlines_llm_response.text)

```json
{
  "KPMG in Singapore blends AI with human touch to revolutionise talent acquisition": 0.95,
  "From fixed to flexible: Why Asia’s top talent expects benefits that move with them": 0.9,
  "Why hiring Gen Z means HR has to rethink the entire talent playbook": 0.85,
  "Scaling challenges in the Philippine startup ecosystem: What founders are up against": 0.8,
  "Organisations in Singapore brace for slower hiring and wage growth": 0.75,
  "Up-skilling for AI: Train for what machines can’t do or learn to work with them?": 0.7,
  "AI adoption is an area of maturity for SMEs, but they have advantage over big corporations: Aicadium’s Robert Young": 0.55,
  "The new builders: AI startups reinventing how Southeast Asia’s enterprises innovate": 0.45,
  "How business leaders can utilise generative AI in employee communications": 0.4,
  "Web3 can absorb SEA’s talent glut, only if education evolves": 0.3,
  "How DITP is connecting Thai startups with Philippine investors": 0.25,
  "Ecosyste

In [ ]:
# Optimized prompt with context and target audience
scoring_headlines_prompt = """Context: You are creating engaging LinkedIn posts for Cygnify, a Talent Acquisition as a Service (TAaaS) company. Cygnify offers unlimited global hires on a fixed monthly subscription, with no placement fees, a plug & play TA team that can onboard in under 5 minutes, and flexible month-to-month plans.

Target audience:
1. Fast-growing startups/scale-ups needing to rapidly build teams globally. They value speed, flexibility, and cost-effective scaling.
2. Mid-to-large enterprises optimizing recruitment costs, seeking predictable expenses and ROI.
3. Companies expanding internationally, needing a partner to navigate global hiring complexities.

Task: Rank the following headlines from most to least engaging for this audience. Assign each headline a numeric score from 0 (not engaging) to 1 (extremely engaging). Return only a JSON object mapping headline to score.

Headlines:
"""

# Append all headlines from the DataFrame
for i, headline in enumerate(scraped_articles_df['Headline'], 1):
    scoring_headlines_prompt += f"{i}. {headline}\n"

# JSON output example to guide the model
scoring_headlines_prompt += "\nJSON output example: {\"Headline text\": 0.75, ...}"

# Call Gemini model
scored_headlines_llm_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=scoring_headlines_prompt
)

# Print LLM output
print(scored_headlines_llm_response.text)

In [ ]:
scored_headlines_llm_response = scored_headlines_llm_response.text.strip()

# Use regex to extract the JSON object from the text
match = re.search(r"\{.*\}", scored_headlines_llm_response, re.DOTALL)
if match:
    json_text = match.group()
    try:
        scores_dict = json.loads(json_text)
        print("JSON successfully parsed!")
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
else:
    print("No JSON found in the response text")

# Add engagement score
scraped_articles_df['EngagementScore'] = scraped_articles_df['Headline'].map(scores_dict)

# Rank headlines by engagement score (highest first)
ranked_articles_df = scraped_articles_df.sort_values(by='EngagementScore', ascending=False).reset_index(drop=True)
ranked_articles_df

JSON successfully parsed!


,Headline,Date,Source,Link,EngagementScore
0,KPMG in Singapore blends AI with human touch t...,2025-08-26,HRM Asia,https://hrmasia.com/kpmg-in-singapore-blends-a...,0.95
1,From fixed to flexible: Why Asia’s top talent ...,2025-08-28,HRM Asia,https://hrmasia.com/from-fixed-to-flexible-why...,0.90
2,Why hiring Gen Z means HR has to rethink the e...,2025-09-01,HRM Asia,https://hrmasia.com/why-hiring-gen-z-means-hr-...,0.85
3,Scaling challenges in the Philippine startup e...,2025-08-29,e27,https://e27.co/the-philippines-challenges-2025...,0.80
4,Organisations in Singapore brace for slower hi...,2025-08-29,HRM Asia,https://hrmasia.com/organisations-in-singapore...,0.75
5,Up-skilling for AI: Train for what machines ca...,2025-08-29,e27,https://e27.co/up-skilling-for-ai-train-for-wh...,0.70
6,"AI adoption is an area of maturity for SMEs, b...",2025-08-28,e27,https://e27.co/ai-adoption-is-an-area-of-matur...,0.55
7,The new builders: AI startups reinventing how ...,2025-09-01,e27,https://e27.co/the-new-builders-ai-startups-re...,0.45
8,How business leaders can utilise generative AI...,2025-08-29,e27,https://e27.co/how-business-leaders-can-utilis...,0.40
9,"Web3 can absorb SEA’s talent glut, only if edu...",2025-09-01,e27,https://e27.co/web3-can-absorb-seas-talent-glu...,0.30


## Step 3: Use Gemini to Generate LinkedIn Captions and Image Prompts

In [ ]:
first_ranked_headline = ranked_articles_df.iloc[0]['Headline']
first_ranked_link = ranked_articles_df.iloc[0]['Link']
print(first_ranked_headline)
print(first_ranked_link)

KPMG in Singapore blends AI with human touch to revolutionise talent acquisition
https://hrmasia.com/kpmg-in-singapore-blends-ai-with-human-touch-to-revolutionise-talent-acquisition/


In [ ]:
# first_ranked_headline = ranked_articles_df.loc[scraped_articles_df['EngagementScore'].idxmax()]['Headline']
# first_ranked_link = ranked_articles_df.loc[ranked_articles_df['EngagementScore'].idxmax()]['Link']
first_ranked_headline = "Why hiring Gen Z means HR has to rethink the entire talent playbook"
first_ranked_link = "https://hrmasia.com/why-hiring-gen-z-means-hr-has-to-rethink-the-entire-talent-playbook/"

# Create a prompt for the LinkedIn post
create_linkedin_caption_prompt = f"""
TASK CONTEXT:
You are helping create engaging LinkedIn posts for Cygnify, a Talent Acquisition as a Service (TAaaS) company.
Cygnify provides unlimited global hires on a fixed-monthly subscription, with no placement fees,
a plug & play TA team that can onboard in under 5 minutes, and flexible month-to-month plans.

TONE CONTEXT:
Always write in Cygnify’s brand voice which is forward-looking, confident, and solution-oriented.
Frame Cygnify as a strategic partner, not a vendor.

TARGET AUDIENCE:
HR leaders, Talent Acquisition professionals, and executives (CEOs, CFOs, COOs)
who care about speed, scalability, predictable spend, and global reach.

KEY VALUE PROPOSITION ELEMENTS:
- Unlimited Hires, No Placement Fees
- Flexible Month-to-Month Plans
- Global Reach & Industry Agnostic
- Plug & Play / Rapid Onboarding
- Full Transparency & Dedicated Support

TARGET AUDIENCE:
1. Fast-growing startups or scale-ups
2. Established mid-to-large enterprises
3. Companies expanding internationally

DETAILED TASK:
Use the following news headline and article link to create content:
**Headline:** "{first_ranked_headline}"
**Article Link:** "{first_ranked_link}"

Use the article content as context to create 2 LinkedIn post drafts for Cygnify’s plug & play TAaaS.
End each post with a CTA that drives engagement or encourages booking a Discovery Call.
Include one engagement question or poll in each post.
Also generate a separate Canva AI image prompt idea for each post (professional, business-relevant, visually striking).

Formats:

### Insight + Stat Hook
- ~70 words
- Hook with a key stat/insight → context → Cygnify tie-in → CTA

### Trend + Cygnify POV
- Include 3 actionable insights from the article
- Trend → Cygnify perspective → advice/CTA

### Paint Point + How Cygnify provides value

"""

linkedin_caption_llm_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=create_linkedin_caption_prompt
)

print(f"The most engaging headline the past week is {first_ranked_headline}. The link to the article is {first_ranked_link}. Based on that: ")
print(linkedin_caption_llm_response.text)

Here are two LinkedIn post drafts for Cygnify, leveraging the provided article content and headline context, crafted in Cygnify's brand voice.

---

### Post 1: Insight + Stat Hook

As organizations navigate shifting economic landscapes and the call for greater efficiency, the need to revolutionize talent acquisition is more urgent than ever. Forward-thinking firms are proving that blending cutting-edge AI with the essential human touch is the future of smart hiring. At Cygnify, we embody this future. Our plug & play TAaaS model brings you a dedicated team, equipped with advanced strategies and technology, ensuring you get unlimited, high-quality hires globally, without the burden of placement fees. We're your strategic partner in building a future-proof workforce.

**CTA:** Ready to transform your TA for efficiency and impact? Book a Discovery Call today!

**Engagement Question:** How are you balancing AI innovation with the crucial human element in your hiring process?

---

**Canva 

## Step 4: Use Canva AI to Generate LinkedIn Image using the Generated Prompts above
https://www.canva.com/ai

Random shades of puple,